# Lista 4

## 1. Wstęp
Na liście 4 celem jest analiza oraz klasyfikacja danych parametrów życiowych płodu z wykorzstanie algorytmów uczenia maszynowego. Zastosowane zostały 4 różne algorytmy. Naive Bayes, Decision Tree, Random Forest, SVR. Porównane zostały skuteczność poszczególnych metod pod kątem miar jakości predykcji. Dodatkowo przeanalizowano wpływ wybranych parametrów na działanie modeli oraz zastosowane techniki przetwarzania danych.


## 2. Implementacja
Generowanie raportu zawierającego informacje o zbiorze wejsciowym, który jest częściowo niepełny oraz zanieczyszczony.

In [83]:
from ydata_profiling import ProfileReport
import pandas as pd
from pathlib import Path

file = Path("csv_data_analyse.html")
if not file.exists():
    df = pd.read_csv("CTG.csv", sep=";", decimal=",")
    profiler = ProfileReport(df, title="Data Profiler")
    profiler.to_file(output_file="csv_data_analyse")

Czyszczenie danych wejściowych

In [84]:
df = pd.read_csv("CTG.csv", sep=";", decimal=",")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.dropna()

Podział na zbiór testowy i uczący. Stosunek zbiorów to 3:1

In [85]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["CLASS"])
y = df["CLASS"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

### Normalizacja oraz Standaryzacja
StandardScaler skaluje globalnie dane tak, aby ich średnia wynosiła 0, a odchylenie standardowe wynosiło 1. Natomiast Normalizer przekształca dane tak aby kazdy wektor cech miał długość równą 1.

In [86]:
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.decomposition import PCA

# data normalization
standardScaler = StandardScaler()
normalizer = Normalizer()

normalizer_x_train = normalizer.fit_transform(X_train)
normalizer_x_test = normalizer.transform(X_test)

scalled_x_train = standardScaler.fit_transform(X_train)
scalled_x_test = standardScaler.transform(X_test)

### Naive Bayes

Naive Bayes to statystyczna technika klasyfikacji oparta na twierdzeniu Bayesa. Jest to jeden z najprostszych algorytmów uczenia nadzorowanego, który cechuje się szybkością, dokładnością i niezawodnością, zwłaszcza przy dużych zbiorach danych.

Algorytm zakłada, że wpływ każdej cechy na przynależność do danej klasy jest niezależny od pozostałych cech nawet jeśli w rzeczywistości są one ze sobą powiązane. To uproszczenie pozwala znacząco zredukować złożoność obliczeniową, dlatego algorytm określa się jako „naiwny”.


In [98]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import GaussianNB

def calculate(clasifier_name, classifer, x_test, y_test, x_train, y_train, file):
    classifer.fit(x_train,y_train)
    pred = classifer.predict(x_test)
    with open(file,"w") as f:
        f.write(f"Accuracy {accuracy_score(y_test, pred)}")
        f.write(classification_report(y_test, pred, zero_division=0))

    print(f"{clasifier_name} accuracy:", accuracy_score(y_test, pred))

gaussians = [
    ("Standard 1e-09",GaussianNB(),"Gaussian/09"),
    ("Standard 1e-12",GaussianNB(var_smoothing=1e-12), "Gaussian/12"),
    ("Standard 1e-12",GaussianNB(var_smoothing=1e-6), "Gaussian/06"),
    ("Standard 0.001",GaussianNB(var_smoothing=0.001), "Gaussian/1")
]
print("GaussianNB")
for name, gaussian, file in gaussians:
    calculate(name,gaussian,normalizer_x_test,y_test,normalizer_x_train,y_train,file+"standard")
    calculate(name,gaussian,scalled_x_test,y_test,scalled_x_train,y_train,file+"scalled")


GaussianNB
Standard 1e-09 accuracy: 0.881578947368421
Standard 1e-09 accuracy: 1.0
Standard 1e-12 accuracy: 0.8609022556390977
Standard 1e-12 accuracy: 1.0
Standard 1e-12 accuracy: 0.8740601503759399
Standard 1e-12 accuracy: 1.0
Standard 0.001 accuracy: 0.37969924812030076
Standard 0.001 accuracy: 1.0


### Decision Trees

Drzewa decyzyjne to struktura przypominająca diagram przepływu, składają się z węzłów reprezentujących
decyzje, gałęzi reprezentujących wynik tych decyzji oraz węzłów liści reprezentujących ostateczne wyniki.
Ważnymi elemntami każdego drzewa jest korzeń (root) reprezentujący zbiór danych i początkową decyzję do
podjęcia. Kolejnym ważnym elementem jest wewnętrzny węzeł (internal node) reprezentują decyzje lub testy
na atrybutach. Gałąź (branch) jest wynikiem decyzji lub testu i prowadzadzi do innego węzła. Każdy node
może mieć wiele gałęzi. Węzęły liści (leaf nodes) czyli końcowewęzły zawierające wynik.

Działanie drzewa decyzyjnego:

1. Drzewo wybiera najlepszy atrybut do podziału danych. Wybierany jest on przy użyciu odpowiednich miar
2. Podział zbioru danych na podstawie wybranego atrybutu
3. Proces jest powtarzany rekurencyjnie dla każdego podzbioru, tworząc nowy węzeł wewnętrzny lub węzeł liścia, aż do spełnienia kryterium zatrzymania
    1. Liczba próbek w węźle jest mniejsza niż określony limit
    2. Głębokość węzła jest większa niż określony limit
    3. Czystość węzła jest większa niż pre-definiowany limit
    4. Wartości predyktorów dla wszystkich rekordów są identyczne, co uniemożliwia wygenerowanie reguły do ich podziału

Metryki uzywane do drzewa podziału drzewa decyzyjnego:
1. Entropy - Mierzy ilość niepewności lub nieczystości w zbiorze danych: $$ 1 − \displaystyle\sum_{i=1}^n (p_i)^2 $$
2. Gini - Służy do przewidywania prawdopodobieństwa nieprawidłowej klasyfikacji losowo wybranego przykładu przez konkretny węzeł:  $$ − \displaystyle\sum_{i=1}^n p_i log_2(p_i) $$
3. Information Gain - Mierzy redukcję entropii lub nieczystości Gini po podziale zbioru danych na podstawie atrybutu : $$ Entropia_{parent} −  \displaystyle\sum_{i=1}^n ( \frac{|D_i|}{|C|}  \times  Entropia(D_i)) $$ 
.

$$ (p_i) - prawdopodobieństwo przypisania przypadku do danej klasy, |D_i|  - podzbiór |D|  po podziale na podstawie atrybutu $$

In [105]:
from sklearn.tree import DecisionTreeClassifier

trees = [
    ("Default", DecisionTreeClassifier(), "Tree/normal"),
    ("Entropy", DecisionTreeClassifier(criterion="entropy"),"Tree/entropy"),
    ("Log_loss", DecisionTreeClassifier(criterion="log_loss"),"Tree/log"), 
    # łagodzenie
    ("Default d=5", DecisionTreeClassifier(max_depth=5), "Tree/normal5"),
    ("Default d=10", DecisionTreeClassifier(max_depth=10), "Tree/normal10"),
    ("Default d=20", DecisionTreeClassifier(max_depth=20), "Tree/normal20"),
    ("Default min", DecisionTreeClassifier(max_depth=10, min_samples_leaf=5, min_samples_split=5), "Tree/min"),
    ("Default alpha" , DecisionTreeClassifier(max_depth=10, ccp_alpha=0.01), "Tree/alpha"),
    ("Default all", DecisionTreeClassifier(max_depth=10, min_samples_leaf=5, min_samples_split=5, ccp_alpha=0.05), "Tree/all"),
]

print("DecisionTreeClassifier")
for name, tree, file in trees:
    calculate(name,tree,normalizer_x_test,y_test,normalizer_x_train,y_train,file+"standard")
    calculate(name,tree,scalled_x_test,y_test,scalled_x_train,y_train,file+"scalled")

DecisionTreeClassifier
Default accuracy: 1.0
Default accuracy: 0.9981203007518797
Entropy accuracy: 1.0
Entropy accuracy: 0.9943609022556391
Log_loss accuracy: 1.0
Log_loss accuracy: 0.9943609022556391
Default d=5 accuracy: 0.881578947368421
Default d=5 accuracy: 0.881578947368421
Default d=10 accuracy: 1.0
Default d=10 accuracy: 0.9962406015037594
Default d=20 accuracy: 1.0
Default d=20 accuracy: 0.9981203007518797
Default min accuracy: 1.0
Default min accuracy: 0.9981203007518797
Default alpha accuracy: 1.0
Default alpha accuracy: 0.9962406015037594
Default all accuracy: 0.881578947368421
Default all accuracy: 0.881578947368421


### Random forest
Random Forest to metoda uczenia się zespołowego, która łączy predykcje z wielu drzew decyzyjnych w celu uzyskania dokładniejszych i stabilnych wyników. 
Random Forest używa się zarówno do Regresji jak i Klasyfikacji. Random Forest składa sie z wielu Drzew Decyzyjnych. Każde drzewo ma tak zwane bootstrap samples.
Bootstrap samples to podzbiór oryginalnego zbioru. Podzbiór ten to losowo wybierane dane. Ważne jest żeby podzbiór ten finalnie zawierał rówież powtórzenia. Oczekuje się 63,2% unikalnych danych.
Dla klażdego subsetu wybiera sie również podzbiór niepowtarzalnych cech. Każde drzewo jest trenowane. Wynik w przypadku regresji jest średnią, natomiast w przypadku klasyfikacji wybierana 
jest odpowiedź występująca najczęściej. Cały proces nazywany jest baggingiem

In [102]:
from sklearn.ensemble import RandomForestClassifier

forests = [
    ("Default", RandomForestClassifier(criterion="gini"), "Random/normal"),
    ("entropy", RandomForestClassifier(criterion="entropy"), "Random/entropy"),
    ("log", RandomForestClassifier(criterion="log_loss"),"Random/log"),
    ("Default depth n = 5", RandomForestClassifier(criterion="gini", max_depth=10), "Random/normal5"),
    ("Default depth n = 15", RandomForestClassifier(criterion="gini", max_depth=15), "Random/normal5"),
]

print("RandomForestClassifier")
for name, forest, file in forests:
    calculate(name,forest,normalizer_x_test,y_test,normalizer_x_train,y_train,file+"standard")
    calculate(name,forest,scalled_x_test,y_test,scalled_x_train,y_train,file+"scalled")

RandomForestClassifier
Default accuracy: 1.0
Default accuracy: 1.0
entropy accuracy: 1.0
entropy accuracy: 1.0
log accuracy: 1.0
log accuracy: 1.0
Default depth n = 5 accuracy: 1.0
Default depth n = 5 accuracy: 1.0
Default depth n = 15 accuracy: 1.0
Default depth n = 15 accuracy: 1.0


### SVR

Support Vector Regression (SVR) jest rozszerzeniem Support Vector Machines. Główną ideą SVR jest wykorzystanie hiperpłaszczyzny w przestrzeni cech danych, która dzieli dane na dwie klasy, reprezentujące różnice między wartościami wyjściowymi. SVR jest algorytmem opartym na odległości, dlatego skalowanie jest ważnym etapem przetwarzania wstępnego, który może poprawić dokładność i stabilność modelu

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import r2_score, root_mean_squared_error

def calculate_regression(model_name, model, x_test, y_test, x_train, y_train, file):
    print(model_name)
    model.fit(x_train, y_train)
    pred = model.predict(x_test)

    with open(file, "w") as f:
        f.write(f"R2 score: {r2_score(y_test, pred)}\n")
        f.write(f"RMSE: {root_mean_squared_error(y_test, pred)}\n")

    print(f"R2 score: {r2_score(y_test, pred):.4f}, RMSE: {root_mean_squared_error(y_test, pred):.4f}")


svrs = [
    ("liner", SVR(kernel="linear"), "SVR/linear"),
    ("poly", SVR(kernel="poly"), "SVR/poly"),
    ("rbf", SVR(kernel="rbf"), "SVR/rbf"),
    ("sigmoid", SVR(kernel="sigmoid"), "SVR/sigmoid"),
]

print("SVR")
for name, svr, file in svrs:
    calculate_regression(name,svr,normalizer_x_test,y_test,normalizer_x_train,y_train,file+"standard")
    calculate_regression(name,svr,scalled_x_test,y_test,scalled_x_train,y_train,file+"scalled")

RandomForestClassifier
liner
R2 score: -0.0362, RMSE: 3.1837
liner
R2 score: 0.9990, RMSE: 0.0999
poly
R2 score: 0.0284, RMSE: 3.0828
poly
R2 score: 0.9619, RMSE: 0.6101
rbf
R2 score: -0.0100, RMSE: 3.1432
rbf
R2 score: 0.9886, RMSE: 0.3345
sigmoid
R2 score: -0.0429, RMSE: 3.1940
sigmoid
R2 score: -1.2070, RMSE: 4.6463


### Analiza danych
Wprzypadku Bayes'a dokładność algorytmu wychodziła znacznie lepiej w przypadku skalowania. W przypadku normalizacji algorytm miał nico mniejsza dokładność.
W przypadku drzew decyzyjnych to normalizacja danych powdowała że dokładność była lepsza. W przypadku Random forest'a dokładnośc wychodziła na podobnym poziomie w obu danych. 
W przypadku SVR wybór odpowiedniego kernela oraz przetworzenie danych wejścowych. W przypadku normalizacji wszystkie kerenle osiągneły bardzo słaby wyniki zarówno w R^2 jak i RMSE. Natomiast standaryzacja jest znacznie lepszym sposobem przetowrzenia danych. W Przyapdku linear kernel dane dopasowały sie niemal idealnie. W przypadku rbf oraz poly dopasowanie również było bardzo dobre oraz dobre. Natomiast najgorszy miał kernel sigmoid. 

W przypadku plików wynikowych dla metody Bayes wyniki były na popziomie 50% dla normalizacji. Prezycja z jaką algorytm działał była mierna. Podobnie jak czułość która również była mierna. Najlepszym sposobem oceny jest f-1 score który jest średnią harmoniczną precision oraz recall. Najlepsze wyniki dla normalizacji były najlepsze dla var_smothing rzędu 1e-07/1e-08. Dla zeskalowanych danych dla każdego smothingu dokładność była rzędu 1. 

W przypadku drzew decyzyjnych dokładnośc była rzędu 1 w przypadku danych które zostały znormalizowane. Dla danych zeskalowanych dokładność była rzędu 0.99. W przypadku kolejnych drzew podczas testów dobór takich parametrów jak max_depth, min_samples_split (minimalna liczba próbek wymagana aby węzeł został podzielony), min_samples_leaf (minimalna liczba próbek wymagana w końcowym węźle), ccp_alpham, pozwalały na manipulowanie trenowaniem, czym wpływały na wynik.
Głowny wpływ na prztrenowania ma dobór parametrów. Parametr cpp_alpha pozwoił w znacznym stopniu manipulować dokładnością. Cost-complexity pruning alpha funkcja ta pozwala zmniejszać koszt oraz upraszcza drzewo, przycinając je. Zapobiega to w głównej mierze przed overfittingiem.    

W przypadku Random Forrest Cieżko stwirdzić jakieś zależności ze względu na dokładność rzędu 1. Ze względu na losowośc tego algorytmu czasem zdarzało się ze dokładność modeli w których kryterium była entrpia, lekko spadał do nawet dokładnośći rzędu 0,95.

## 3. Podsumowanie
Ze wzgędu na poprzednie doświadczenie w jedym projekcie z SVR oraz Random Forrest ćwiczenie było dużo łatwiejsze w realizacji. 
W przypadku każdeg oalgorymtu należy uważnie dobierać parametry oraz dane. Sama zabawa z prostymi algorytmami uczenia maszynowego wymaga nawet minimalnego przygotowania oraz wiedzy aby umyślnie dobierać parametry. Jak zostało zaprezentowane powyżej nawet dobne zmiany moga prowadzić do zmian. Również zbiór danych oraz odpowiednie przygotowanie go może wpłynąć na wynik oraz dokładność modelu. Kolejnym ważnym aspektem na który należy zwracać uwagę jest również przetrenowanie modeli, co również wiąże się z dobieraniem odpowienich parametrów algorytmów.   